# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager
import time
#pagina web que vamos a hacer el scraping
url='https://www.latamairlines.com/ar/es/ofertas-vuelos?origin=ASU&inbound=null&outbound=2022-12-01T15%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

## Demora Dinamica
<p>Para evitar fallos de carga de la página y de mensajes emergente que no dejen realizar el scraper, se implementan demoras dinámicas, estas a diferencia de la demora estática brindan un mejor rendimiento. Ya que no tienen que esperar el tiempo estático sino que esperan a que se cargue la página y si tarda menos del tiempo estipulado, empieza a hacer el scraper.</p>

 Lo mejor sería esperar a que la página termine de cargar y luego recuperar los elementos. para ello importamos varias librerias

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [3]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas= vuelo.find_elements('xpath','.//ol[@class="sc-iAVDmT DwgCo"]/li[@class="sc-jjgyjb buqrVI"]')
    precios=[]
    for tarifa in tarifas:
        #buscamos en cada pocision de la tarifa los siquientes elementos
        nombre = tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[1]/span[@class="sc-gBSKhj chxCgG"]').text
        moneda= tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[3]//span[contains(@class,"currency")]').text
        valor= tarifa.find_element('xpath','.//div[@class="sc-ieSwJA gfCbFb"]/div[3]//span[@class="sc-ihiiSJ AoaXI"]').text
        #guardo los valores que obtengo en un diccionario
        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
        #guradamos nuestro diccionario con los datos de tarifa a nuestra lia de precio
        precios.append(dict_tarifa)
        # print(dict_tarifa)
    return precios

In [4]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de 
    las escalas de cada vuelo
    '''

    segmentos= vuelo.find_elements('xpath','//div[@class="MuiDialogContent-root sc-fjdhpX jvVjjf"]//section[@class="sc-fEVUGC gIelIH"]')
    datos_escalas=[]    
    for segmento in segmentos:
        #origen
        salida=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[1]').text
        #hora de salida
        hora_salida=segmento.find_elements('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[2]').text
        #destino
        destino_segmento=segmento.find_element('xpath','.//div[@class="sc-hAcydR ikJhgn"]/div[@class="iataCode"]/span[1]').text
        #Hora de llegada
        hora_llegada=segmento.find_element('xpath','.//div[@class="sc-hAcydR ikJhgn"]/div[@class="iataCode"]/span[2]').text
        #Duracion del vuelo
        duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-BOulX kmIWrd"]/span[2]').text
        #Numero de vuelo
        numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//div[@class="sc-bscRGj iggkUa airline-wrapper"]').text
        #Modelo de avion
        modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//span[@class="airplane-code"]').text
        
        #el último segmento no tendrá escala por que es el destino final del vuelo
        if segmento != segmentos[-1]:    
            #Duracion escala
            duracion_escala_vuelo=segmento.find_element('xpath','//section[@class="sc-fEVUGC gAwpmW"]//span[@class="time"]').text
        else:
            duracion_escala_vuelo=''
        
        # Armo un diccionario para almacenar los datos
        datos_escalas_dict={
            'salida':salida,
            'hora_salida':hora_salida,
            'destino':destino_segmento,
            'hora_llega':hora_llegada,
            'duracion':duracion_segmento,
            'numero_vuelo':numero_vuelo_segmento,
            'modelo_avion':modelo_avion_segmento,
            'duracion_escala':duracion_escala_vuelo
        }
        datos_escalas.append(datos_escalas_dict)

    return datos_escalas

In [5]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    #hora de salida
    hora_salida=vuelo.find_element('xpath','//div[@class="sc-ixltIz dfdfxH flight-information"]/span[1]').text
    #hora de llegada
    hora_llegada=vuelo.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
    # Duracion del vuelo
    duracion_vuelo= vuelo.find_element('xpath','.//div[2]/span[2]').text

    tiempos_vuelo_dict={
        'hora_salida':hora_salida,
        'hora_llegada':hora_llegada,
        'duracion_vuelo':duracion_vuelo
    }
    return tiempos_vuelo_dict

In [6]:
def obtener_info(driver):
    #Usaremos el Xpath para obtener la lista de vuelos
    vuelos = driver.find_elements('xpath','//ol/li')
    print(f'Se encontaron {len(vuelos)} vuelos.')
    print(f'Iniciando Scraping...')
    info=[]
    contador=0
    for vuelo in vuelos:
        #obtener los tiempos generales de cada vuelo
        tiempos=obtener_tiempos(vuelo)
        #esperamos que cargue el contenido con demora dinamica
        delay=10
        try:
            #introducimos demora inteligente
            #clickeamos sobe el link escalas para que se habra el modal
            vuelo.find_element('xpath','//div[@class="sc-fYAFcb kdctDt"]//a').click()
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, './/div[@id="itinerary-modal-0-dialog-content"]')))
            print('El modal del vuelo terminó de cargar')
            escalas=obtener_datos_escalas(vuelo)
        except TimeoutException:
            print("La página tardó demasiado en cargar")
        
        #cerramos el modal
        driver.find_element('xpath','//*[@id="itinerary-modal-0-dialog-close"]').click()
        
        try:
            #introducimos demora inteligente
            #clickeamos el vuelo para ver los precios
            vuelo.click()
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH,'.//ol[@class="sc-iAVDmT DwgCo"]')))
            print('La tabla de precios terminó de cargar')
            precios=obtener_precios(vuelo)
        except TimeoutException:
            print("La página tardó demasiado en cargar")
        #click nuevamente para cerrar los precios que se desplegaron
        vuelo.click()

        info.append({'precios':precios,'tiempos':tiempos,'escalas':escalas})
        contador+=1
        print(f'Analizando {contador} de {len(vuelos)} vuelos')

    return info

In [7]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)
driver.get(url)
time.sleep(5)
def close_dialog():
    '''
    Función que cierra los dialog o popups de la página
    '''
    dialogs = driver.find_elements(By.XPATH, '//div[contains(@class,"MuiDialog-container")]')
    if len(dialogs):
        dialogs[0].find_element(By.XPATH, './/button[contains(@class,"Dialog__CloseButton")]').click()
close_dialog()

try:
    # introducir demora inteligente
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="WrapperBodyFlights"]/ol/li')))
    print('La página terminó de cargar')
    obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
finally:
    print('error')
    #driver.close()

C:\Users\mario\AppData\Local\Temp\ipykernel_9192\3696530526.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


La página terminó de cargar
Se encontaron 30 vuelos.
Iniciando Scraping...
El modal del vuelo terminó de cargar
error


NoSuchElementException: Message: Unable to locate element: .//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[2]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:280:16


Paso 4: cerrar el navegador

In [ ]:
driver.close()